## KDS75

In [1]:
# Load modules
%pylab inline
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import seaborn as sns

from netCDF4 import Dataset # to work with NetCDF files
import numpy as np
from mpl_toolkits.basemap import Basemap # plot on map projections
from os.path import expanduser
home = expanduser("~") # Get users home directory
import os # operating system interface
import scipy as sp
import xarray as xr
from xarray.ufuncs import square, sqrt
import scipy.interpolate
import scipy.ndimage




Populating the interactive namespace from numpy and matplotlib
netcdf_index loaded.


In [2]:
from joblib import Memory

memory = Memory(cachedir='/g/data1/v45/cosima-cookbook/', verbose=0)

In [3]:
cc.start_cluster(diagnostics_port=8787)

If the link to the dashboard below doesn't work, run this command on a local terminal to set up a SSH tunnel:

  ssh -N -L 8787:127.0.0.1:8787 vdi-n14.nci.org.au -l pas561


Client Scheduler: tcp://127.0.0.1:44357 Dashboard: http://127.0.0.1:8787,Cluster Workers: 4 Cores: 8 Memory: 24.00 GB


In [4]:
#A = xr.DataArray(np.arange(100))
#A.cumsum(axis=0)
print (xr.__version__)
#my xarray version is old, no cumsum. Should be version 0.9.2 or newer?

0.10.0


In [5]:
NYFCntrlDir = '/g/data3/hh5/tmp/pas561/kds75_cp/'
NYFPertDir = '/g/data3/hh5/tmp/pas561/kds75_wp2/'


In [6]:
expts = cc.get_experiments('/g/data3/hh5/tmp/pas561/')
display(expts)


[]

In [8]:
#this fails due to memory errors ... takes forever
Ocean_File = os.path.join(NYFPertDir,'output4*/ice_month.nc')
pert_i=xr.open_mfdataset(Ocean_File,engine='netcdf4',concat_dim='time',decode_times=False)
print(pert_i.CN.shape)
print(pert_i.CN[:,:,:,:].sum('ct').shape)

KeyboardInterrupt: 

In [11]:
#if i fist ncrcat the CN data into an netcdf file, it works and is very fast
Ocean_File = os.path.join(NYFPertDir,'CN.cat.nc')
pert_i=xr.open_mfdataset(Ocean_File,engine='netcdf4',concat_dim='time',decode_times=False)
print pert_i.CN.shape
print pert_i.CN[:,:,:,:].sum('ct').shape

(87, 5, 2700, 3600)
(87, 2700, 3600)


In [7]:
#even more painful is calculating the overturning
Ocean_File = os.path.join(NYFPertDir,'output4*/ocean.nc')
pert=xr.open_mfdataset(Ocean_File,engine='netcdf4',concat_dim='time',decode_times=False)
print(pert.ty_trans_rho.shape)

pert_gmoc=pert.ty_trans_rho[:,:,:,:].sum('grid_xt_ocean').cumsum('potrho').load()
print(pert_gmoc.shape)

of=os.path.join(NYFPertDir,'gmoc_adv_rho.npy')
a=np.save(of,pert_gmoc)



(29, 75, 2700, 3600)


MemoryError: 

In [ ]:
#instead I can opt for the slow, but successful version below:
print pert.ty_trans_rho.shape[0]

pert_gmoc=np.zeros((pert.ty_trans_rho.shape[0],pert.ty_trans_rho.shape[1],pert.ty_trans_rho.shape[2]))
print pert_gmoc.shape

for t in range(pert.ty_trans.shape[0]):
    print t
    pert_gmoc[t,:,:]=pert.ty_trans_rho[t,:,:,:].sum('grid_xt_ocean').cumsum('potrho').load()

print pert_gmoc.shape

of=os.path.join(NYFPertDir,'gmoc_adv_rho.npy')
a=np.save(of,pert_gmoc)
